In [ ]:
using Distributions
using Plots
using Colors
using ColorSchemes
using JLD
using SpecialFunctions
using LinearAlgebra

pyplot()

In [ ]:
using PyPlot
plt = PyPlot

SMALL_SIZE = 12
MEDIUM_SIZE = 13
BIGGER_SIZE = 13

plt.rc("font", size=SMALL_SIZE)          # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)    # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
function rect_lower(N, d; n=20)
    quantile(Normal(), (1 + (n/N)^(1/(d)))/2)
end

function rect_upper(N, d; n=1)
    sqrt(quantile(Chisq(d), 1-n/N )/d)
end

function sphere_upper(N, d; n=1)
    sqrt(quantile(Chisq(d), 1-n/N ))
end

function sphere_lower(N, d; n=20)
    sqrt(quantile(Chisq(d), n/N ))
end

# Generate Experiments: 

In [ ]:
mask_rectangular(samples, window) = prod(abs.(samples) .< window, dims=1)[1,:]
mask_spherical(samples, window) = [sum(i.^2)<window^2 for i in eachcol(samples)]
volume_rectangular(n, window) = (2*window)^n
volume_spherical(n, window) = (pi^(n/2))/(gamma(n/2 + 1))*window^n

function estimate_integral(log_lik, V, N_omega, N_delta)
    
    x = 1 ./ log_lik
    x_variance = var(x)/N_delta
    x_mean = mean(x)
    bias_x = x_variance / x_mean^2
    hm_estimate = V/x_mean
    
    r = N_delta / N_omega
    bias_r = (1-r)/N_delta
   
    return hm_estimate, r, bias_r, bias_x
end

function generate_experiments(mask_function, volume_function, dim_range, window_size, n_experiments; n_samples=10^6, sigma=1)

    x_estimate = Vector{Float64}()
    r_estimate = Vector{Float64}()
    bias_x =  Vector{Float64}()
    bias_r =  Vector{Float64}()
    parameters = []
    
    for dim_run in dim_range
        @show "Dim:", dim_run
        
        for exp_run in 1:n_experiments
            
            distribution = MvNormal(dim_run, sigma)
#             distribution = MvNormal(ones(dim_run), Matrix{Float64}(I, dim_run, dim_run))
            smpl = rand(distribution, n_samples)
            log_lik = [pdf(distribution, i) for i in eachcol(smpl)];
    
            for window in window_size

                mask = mask_function(smpl, window)

                smpl_run = smpl[:,mask]
                log_lik_run = log_lik[mask]
                
                V = volume_function(dim_run, window)
                
                x_run, r_run, b_r_run, b_x_run = estimate_integral(log_lik_run, V, length(smpl), sum(mask))
                
                push!(x_estimate, x_run)
                push!(r_estimate, r_run)
                push!(bias_x, b_x_run)
                push!(bias_r, b_r_run)
                push!(parameters, (dim_run, exp_run, window))
            
            end
        end
    end
    
    return(x_estimate, r_estimate, bias_x, bias_r, parameters)
end  

# Generare Samples: 

In [ ]:
N_dim = 2
sigma = 1
N_samples = 10^6
distribution = MvNormal(N_dim, sigma)
# distribution = MvNormal(ones(N_dim), Matrix{Float64}(I, N_dim, N_dim))
smpl = rand(distribution, N_samples)
log_lik = [pdf(distribution, i) for i in eachcol(smpl)];

In [ ]:
histogram2d(smpl[1,:], smpl[2,:], frame=true, colorbar=false)

# Single run (hypercubes):

In [ ]:
window = 2.0

V = (2*window)^N_dim

mask = abs.(smpl) .< window

mask = prod(mask, dims=1)[1,:]

tmp = smpl[:,mask]

@show V, sum(mask)

histogram2d(tmp[1,:], tmp[2,:], frame=true, colorbar=false, bins=100)


In [ ]:
x_run, r_run, b_r_run, b_x_run = estimate_integral(log_lik[mask], V, length(smpl), sum(mask))

In [ ]:
int_tmp = (x_run/r_run)/N_dim

In [ ]:
int_tmp*(1 - b_r_run - b_x_run)

# Single run (hypersphere): 

In [ ]:
r = 2.0

vol(N, R) = (pi^(N/2))/(gamma(N/2 + 1))*R^N

V = vol(N_dim, r)

mask = [sum(i.^2)<r^2 for i in eachcol(smpl)]

tmp = smpl[:,mask]

@show length(smpl), sum(mask), V

histogram2d(tmp[1,:], tmp[2,:], frame=true, colorbar=false, bins=100)

In [ ]:
x_run, r_run, b_r_run, b_x_run = estimate_integral(log_lik[mask], V, length(smpl), sum(mask))

In [ ]:
int_tmp = (x_run/r_run)/N_dim

In [ ]:
int_tmp*(1 - b_r_run - b_x_run)

# Rectangular Volumes:

In [ ]:
dim_range = 2:30 #2:50
window_size = range(0.1, stop=10.0, length=30)
n_experiments = 50;

In [ ]:
@time x_estimate_1, r_estimate_1, bias_x_1, bias_r_1, parameters_1 = generate_experiments(mask_rectangular, volume_rectangular, dim_range, window_size, n_experiments, n_samples=10^4);

In [ ]:
# ****
# save samples
# ****

# save("../../AHMI_publication/Revtangules_vs_Spheres/rect-2.jld", 
#     "x_estimate_1", x_estimate_1,
#     "r_estimate_1", r_estimate_1,
#     "bias_x_1", bias_x_1,
#     "bias_r_1", bias_r_1,
#     "parameters_1", parameters_1)

# ****
# load samples
# ****

# saved_1 = load("../../AHMI_publication/Revtangules_vs_Spheres/rect-2.jld");
# x_estimate_1 = saved_1["x_estimate_1"]
# r_estimate_1 = saved_1["r_estimate_1"]
# bias_x_1 = saved_1["bias_x_1"]
# bias_r_1 = saved_1["bias_r_1"]
# parameters_1 = saved_1["parameters_1"];

In [ ]:
int_estimate_biased_1 = x_estimate_1 ./(r_estimate_1)
int_estimate_unbiased_1 = int_estimate_biased_1 .* (1 .- bias_r_1 .- bias_x_1 ) #.- bias_x

aver_int_estimate_biased_1 = mean(reshape(int_estimate_biased_1, length(window_size), n_experiments, (dim_range[end] - dim_range[1]+1)), dims=2)[:,1,:]
aver_int_estimate_unbiased_1 = mean(reshape(int_estimate_unbiased_1, length(window_size), n_experiments, (dim_range[end] - dim_range[1]+1)), dims=2)[:,1,:]

std_int_estimate_biased_1 = std(reshape(int_estimate_biased_1, length(window_size), n_experiments, (dim_range[end] - dim_range[1]+1)), dims=2)[:,1,:] 
std_int_estimate_unbiased_1 = std(reshape(int_estimate_unbiased_1, length(window_size), n_experiments, (dim_range[end] - dim_range[1]+1)), dims=2)[:,1,:];

In [ ]:
int_true_1 = ones(size(std_int_estimate_biased_1)) .* collect(dim_range)';

In [ ]:
plot_ratio_biased_1 = aver_int_estimate_biased_1./int_true_1;
plot_std_ratio_biased_1 = std_int_estimate_biased_1./int_true_1;

In [ ]:
plot_ratio_unbiased_1 = aver_int_estimate_unbiased_1./int_true_1;
plot_std_ratio_unbiased_1 = std_int_estimate_unbiased_1./int_true_1;

# Spherical Volumes:

In [ ]:
@time x_estimate_2, r_estimate_2, bias_x_2, bias_r_2, parameters_2 = generate_experiments(mask_spherical, volume_spherical, dim_range, window_size, n_experiments, n_samples=10^4);

In [ ]:
# ****
# save samples
# ****

# save("../../AHMI_publication/Revtangules_vs_Spheres/spheres-2.jld", 
#     "x_estimate_2", x_estimate_2,
#     "r_estimate_2", r_estimate_2,
#     "bias_x_2", bias_x_2,
#     "bias_r_2", bias_r_2,
#     "parameters_2", parameters_2)

# ****
# load samples
# ****

# saved_2 = load("../../AHMI_publication/Revtangules_vs_Spheres/spheres-2.jld");

# x_estimate_2 = saved_2["x_estimate_2"]
# r_estimate_2 = saved_2["r_estimate_2"]
# bias_x_2 = saved_2["bias_x_2"]
# bias_r_2 = saved_2["bias_r_2"]
# parameters_2 = saved_2["parameters_2"];

In [ ]:
int_estimate_biased_2 = x_estimate_2 ./(r_estimate_2)
int_estimate_unbiased_2 = int_estimate_biased_2 .* (1 .- bias_r_2 .- bias_x_2 ) #.- bias_x

aver_int_estimate_biased_2 = mean(reshape(int_estimate_biased_2, length(window_size), n_experiments, (dim_range[end] - dim_range[1]+1)), dims=2)[:,1,:]
aver_int_estimate_unbiased_2 = mean(reshape(int_estimate_unbiased_2, length(window_size), n_experiments, (dim_range[end] - dim_range[1]+1)), dims=2)[:,1,:]

std_int_estimate_biased_2 = std(reshape(int_estimate_biased_2, length(window_size), n_experiments, (dim_range[end] - dim_range[1]+1)), dims=2)[:,1,:] 
std_int_estimate_unbiased_2 = std(reshape(int_estimate_unbiased_2, length(window_size), n_experiments, (dim_range[end] - dim_range[1]+1)), dims=2)[:,1,:];

In [ ]:
int_true_2 = ones(size(std_int_estimate_biased_2)) .* collect(dim_range)';

In [ ]:
plot_ratio_biased_2 = aver_int_estimate_biased_2./int_true_2;
plot_std_ratio_biased_2 = std_int_estimate_biased_2./int_true_2;

In [ ]:
plot_ratio_unbiased_2 = aver_int_estimate_unbiased_2./int_true_2;
plot_std_ratio_unbiased_2 = std_int_estimate_unbiased_2./int_true_2;

# Combined Plot: 

In [ ]:
range_v = [0.97, 1.03]


fig, ax = plt.subplots(ncols=3,figsize=(13,6), gridspec_kw=Dict("width_ratios"=>[1,1, 0.03]),)

im1 = ax[1].pcolormesh(dim_range, window_size, plot_ratio_unbiased_1, vmin=range_v[1], vmax=range_v[2], cmap="RdYlBu_r")
im2 = ax[2].pcolormesh(dim_range, window_size, plot_ratio_unbiased_2, vmin=range_v[1], vmax=range_v[2], cmap="RdYlBu_r")

ax[2].get_yaxis().set_visible(false) 

ax[1].set_xlabel("# Dimensions")
ax[2].set_xlabel("# Dimensions")
ax[1].set_ylabel("a")

fig.colorbar(im1, cax=ax[3], ax=[ax[1],ax[2]])


In [ ]:
range_v = [0.97, 1.03]

rect_low_edge = rect_lower.(10^6, collect(dim_range))
rect_upper_edge = rect_upper.(10^6, collect(dim_range), n=1)

spheres_low_edge = sphere_lower.(10^6, collect(dim_range))
spheres_upper_edge = sphere_upper.(10^6, collect(dim_range), n=1)


fig, ax = plt.subplots(ncols=3,figsize=(13,6), gridspec_kw=Dict("width_ratios"=>[1,1, 0.03]),)

im1 = ax[1].pcolormesh(dim_range, window_size, plot_ratio_unbiased_1, vmin=range_v[1], vmax=range_v[2], cmap="RdYlBu_r")
ax[1].plot(dim_range, rect_low_edge, c="cyan", lw=3, alpha=1)
ax[1].plot(dim_range, rect_upper_edge, c="magenta", lw=3, alpha=1)

ax[1].set_ylim(0.1, 10)
ax[1].set_xlim(2, 50)

im2 = ax[2].pcolormesh(dim_range, window_size, plot_ratio_unbiased_2, vmin=range_v[1], vmax=range_v[2], cmap="RdYlBu_r")
ax[2].plot(dim_range, spheres_low_edge, c="cyan", lw=3, alpha=1)
ax[2].plot(dim_range, spheres_upper_edge, c="magenta", lw=3, alpha=1)


ax[2].set_ylim(0.1, 10)
ax[2].set_xlim(2, 50)

ax[2].get_yaxis().set_visible(false) 

ax[1].set_xlabel("# Dimensions")
ax[2].set_xlabel("# Dimensions")
ax[1].set_ylabel("a")

fig.colorbar(im1, cax=ax[3], ax=[ax[1],ax[2]])